In [1]:
from pathlib import Path
import torch
from gelos.gelosdatamodule import GELOSDataModule
import yaml
from gelos import config
from lightning.pytorch import Trainer
from pathlib import Path
from tqdm import tqdm
from gelos.config import PROJ_ROOT, PROCESSED_DATA_DIR, DATA_VERSION, RAW_DATA_DIR
from terratorch.tasks import EmbeddingGenerationTask


2025-11-26 04:19:29.101 | INFO     | gelos.config:<module>:11 - PROJ_ROOT path is: /app
/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
class LenientEmbeddingGenerationTask(EmbeddingGenerationTask):
    def check_file_ids(self, file_ids, x):
        return

def generate_embeddings(yaml_path: Path) -> None:

    with open(yaml_path, "r") as f:
        yaml_config = yaml.safe_load(f)
    
    print(yaml.dump(yaml_config))

    model_name = yaml_config['model']['init_args']['model']
    output_dir = PROCESSED_DATA_DIR / DATA_VERSION / model_name
    output_dir.mkdir(exist_ok=True, parents=True)
    data_root = RAW_DATA_DIR / DATA_VERSION

    # add variables to yaml config so it can be passed to classes
    yaml_config['data']['init_args']['data_root'] = data_root
    yaml_config['model']['init_args']['output_dir'] = output_dir

    gelos_datamodule = GELOSDataModule(**yaml_config['data']['init_args'])
    task = LenientEmbeddingGenerationTask(**yaml_config['model']['init_args'])

    device = 'gpu' if torch.cuda.is_available() else 'cpu'
    trainer = Trainer(accelerator=device, devices=1)

    trainer.predict(model=task, datamodule=gelos_datamodule)


In [3]:
yaml_config_directory = PROJ_ROOT / 'gelos' / 'configs'
yaml_paths = list(yaml_config_directory.glob('*.yaml'))
print(yaml_paths)

[PosixPath('/app/gelos/configs/terramind_embedding_generation.yaml'), PosixPath('/app/gelos/configs/prithvi_eo_300m_embedding_generation.yaml')]


In [ ]:
yaml_path = yaml_paths[0]
generate_embeddings(yaml_path)